### using static (non-trainable) glove vectors with convolutional layers to classify newsgroup_20 documents

https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html


In [18]:
import os
import sys
import numpy as np
import tensorflow as tf

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, Dropout, Embedding, GlobalMaxPooling1D, MaxPooling1D
from keras.models import Model,Sequential

module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)

from helpers import (files_helper, 
                     metrics_helper, 
                     tags_helper,
                     texts_helper)


In [5]:
SEED=np.random.randint(1,1000)
SEED

809

In [6]:
np.random.seed(SEED)

In [7]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 1000
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM=100
LABELS_MIN_DOC_COUNT = int(10)
BATCH_SIZE=32
NUM_EPOCHS=10
TOKENIZER_FILTERS='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n' # default keras filters

In [8]:
embeddings_index = files_helper.read_glove_wiki(d=EMBEDDING_DIM)

In [9]:
texts,labels_index,labels_0 = files_helper.read_20_newgroup_files()       

In [10]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS,
                      filters=TOKENIZER_FILTERS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
inverse_word_index = texts_helper.build_inverse_word_index(word_index)

In [11]:
len(word_index),len(labels_index)

(174074, 20)

In [12]:
# [inverse_word_index[idx] for idx in sequences[0]]

In [13]:
data = pad_sequences(sequences,maxlen=MAX_SEQUENCE_LENGTH)

labels_1 = to_categorical(np.asarray(labels_0))

indices = np.arange(data.shape[0])
np.random.shuffle(indices)

data = data[indices]
labels_2 = labels_1[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

X_train = data[:-num_validation_samples]
y_train = labels_2[:-num_validation_samples]
X_val = data[-num_validation_samples:]
y_val = labels_2[-num_validation_samples:]

data.shape,labels_2.shape

((19997, 1000), (19997, 20))

In [14]:
embedding_matrix = np.zeros((len(word_index)+1,EMBEDDING_DIM))

for word,i in word_index.items():
    
    if i >= MAX_NB_WORDS:
        continue
    
    embedding_vector = embeddings_index.get(word)
    
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [15]:
embedding_matrix.shape

(174075, 100)

In [36]:
model = Sequential()

model.add(Embedding(len(word_index)+1,
                           EMBEDDING_DIM,
                           weights=[embedding_matrix],
                           input_length=MAX_SEQUENCE_LENGTH,
                           trainable = False))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(Dropout(0.5))
model.add(GlobalMaxPooling1D())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(labels_index),activation='softmax'))

model.compile(loss ='categorical_crossentropy',
             optimizer='rmsprop',
             metrics=['acc'])

In [37]:
%%time

model.fit(X_train,y_train, validation_data=(X_val, y_val),
         epochs=NUM_EPOCHS, batch_size=BATCH_SIZE)

Train on 15998 samples, validate on 3999 samples
Epoch 1/10
15998/15998 [==============================] - 19s - loss: 2.3530 - acc: 0.2025 - val_loss: 1.7834 - val_acc: 0.3931
Epoch 2/10
15998/15998 [==============================] - 18s - loss: 1.5438 - acc: 0.4537 - val_loss: 1.3441 - val_acc: 0.5629
Epoch 3/10
15998/15998 [==============================] - 17s - loss: 1.1821 - acc: 0.5815 - val_loss: 1.0968 - val_acc: 0.6552
Epoch 4/10
15998/15998 [==============================] - 17s - loss: 0.9769 - acc: 0.6569 - val_loss: 0.9752 - val_acc: 0.6757
Epoch 5/10
15998/15998 [==============================] - 18s - loss: 0.8510 - acc: 0.7071 - val_loss: 0.9267 - val_acc: 0.7049
Epoch 6/10
15998/15998 [==============================] - 18s - loss: 0.7436 - acc: 0.7508 - val_loss: 0.9389 - val_acc: 0.6999
Epoch 7/10
15998/15998 [==============================] - 18s - loss: 0.6705 - acc: 0.7713 - val_loss: 0.9315 - val_acc: 0.7017
Epoch 8/10
15998/15998 [==============================]